In [1]:
from skimage.transform import resize
from itertools import product
from collections import OrderedDict
from tqdm.notebook import tqdm
import os
import json

import torch
from torch.utils.data import DataLoader
from torchvision import transforms, utils, models
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from sklearn.model_selection import train_test_split

from Run import RunBuilder as RB
from Run import RunManager as RM
from DataLoading import UdacityDataset as UD
from DataLoading import ConsecutiveBatchSampler as CB

from model import TransferLearning as TLearn

%run Visualization/Visualization.ipynb

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cpu


# Data loading

In [2]:
def read_json(data_dir):
    json_file_name = "dataset.json"
    json_file_path = os.path.join(data_dir, json_file_name)
    print(json_file_path)
    if os.path.exists(json_file_path):
        with open(json_file_path, 'r') as file:
            return json.load(file)
    else:
        print(f"The JSON file '{json_file_name}' does not exist in the directory '{data_dir}'.")
        return None

# Model Training / Loading

In [8]:
from collections import OrderedDict


def initialize_parameters():
    return OrderedDict(
        file=['3DCNN_Paper'],
        learning_rate=[0.001],
        batch_size=[5],
        seq_len=[5],
        num_workers=[0],
    )


def setup_data_loaders(data_dir, run):
    json_dataset = read_json(data_dir)
    dataset_loader = UD.UdacityDataset(dataset=json_dataset)
    training_set, validation_set = dataset_loader.split_dataset()

    training_cbs = CB.ConsecutiveBatchSampler(
        data_source=training_set, batch_size=run.batch_size, shuffle=True, drop_last=False, seq_len=run.seq_len
    )
    validation_cbs = CB.ConsecutiveBatchSampler(
        data_source=validation_set, batch_size=run.batch_size, shuffle=False, drop_last=False, seq_len=run.seq_len
    )

    training_loader = DataLoader(
        training_set, sampler=training_cbs, num_workers=run.num_workers, collate_fn=identity_collate
    )
    validation_loader = DataLoader(
        validation_set, sampler=validation_cbs, num_workers=run.num_workers, collate_fn=identity_collate
    )

    return training_loader, validation_loader


def identity_collate(batch):
    return batch


def train_one_epoch(epoch, training_loader, network, optimizer, device):
    print(f"Epoch {epoch + 1}/10 - Training")
    for training_sample in tqdm(training_loader, total=int(len(training_loader.dataset) / training_loader.batch_size)):
        training_sample['image'] = torch.Tensor(
            resize(training_sample['image'], (training_loader.batch_size, training_loader.seq_len, 3, 120, 320),
                   anti_aliasing=True))
        training_sample['image'] = training_sample['image'].permute(0, 2, 1, 3, 4)

        # Get data and train model
        images, angles = training_sample['images'], training_sample['angles']
        images, angles = images.to(device), angles.to(device)

        predictions = network(images).squeeze().permute(1, 0).to(device)

        # Compute loss and update weights
        loss = F.mse_loss(predictions, angles)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()


def validate_model(validation_loader, network, device):
    network.eval()
    total_validation_loss = 0
    num_samples = 0
    for validation_sample in tqdm(validation_loader,
                                  total=int(len(validation_loader.dataset) / validation_loader.batch_size)):
        validation_sample['image'] = torch.Tensor(
            resize(validation_sample['image'], (validation_loader.batch_size, validation_loader.seq_len, 3, 120, 320),
                   anti_aliasing=True))
        validation_sample['image'] = validation_sample['image'].permute(0, 2, 1, 3, 4)

        images, angles = validation_sample['images'], validation_sample['angles']
        images, angles = images.to(device), angles.to(device)

        predictions = network(images).squeeze().permute(1, 0).to(device)
        if angles.shape[0] != predictions.shape[0]:
            predictions = predictions[-angles.shape[0]:]

        validation_loss = F.mse_loss(predictions, angles)
        total_validation_loss += validation_loss.item() * images.size(0)
        num_samples += images.size(0)

    return total_validation_loss / num_samples


def main():
    parameters = initialize_parameters()
    m = RM.RunManager()
    script_dir = os.getcwd()
    data_dir = os.path.join(script_dir, 'data')

    for run in RB.RunBuilder.get_runs(parameters):
        print(f"Starting run: {run}")
        network = TLearn.TLearning().to(device)
        optimizer = optim.Adam(network.parameters(), lr=run.learning_rate, betas=(0.9, 0.999), eps=1e-08,
                               weight_decay=0.001)

        training_loader, validation_loader = setup_data_loaders(data_dir, run)
        m.begin_run(run, network, training_loader)

        for epoch in range(10):
            m.begin_epoch()
            train_one_epoch(epoch, training_loader, network, optimizer, device)
            validation_loss = validate_model(validation_loader, network, device)
            m.track_loss(validation_loss, 'validation')
            print(f"Validation Loss: {validation_loss:.4f}")
            m.end_epoch()
            torch.save(network.state_dict(), f"saved_models/CNN3D/epoch-{epoch}.pth")

        m.end_run()
    m.save('result')


if __name__ == "__main__":
    main()

Starting run: Run(file='3DCNN_Paper', learning_rate=0.001, batch_size=5, seq_len=5, num_workers=0)
D:\PythonProjects\Self-Driving-Car\data\dataset.json


Loading data:   0%|          | 0/5 [00:00<?, ?batch/s]


AttributeError: 'list' object has no attribute 'keys'